In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt

## Preprocessing

In [2]:
# Read the charity_data.csv from the provided cloud URL.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'STATUS' and 'SPECIAL_CONSIDERATIONS'.
# In entire dataset STATUS = 1 for all rows and SPECIAL_CONSIDERATION = No
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [4]:
# Determine the number of unique values in each column.
unique_values = application_df.nunique()

# Display the unique value counts for each column
print(unique_values)

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Display the value counts 
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Set the cutoff value to 200 and create the list
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[
    application_df['APPLICATION_TYPE'].value_counts() < 500
].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_type_counts = application_df['CLASSIFICATION'].value_counts()
classification_type_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts to identify and replace with "Other" 
with pd.option_context('display.max_rows', None):
    print(classification_type_counts[classification_type_counts >= 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
C2190        1
C4200        1
C2600        1
C5200        1
C1370        1
C1248        1
C6100        1
C1820        1
C1900        1
C1236        1
C3700        1
C2570        1
C1580        1
C1245        1
C2500        1
C1570        1
C1283        1
C2380        1
C1732        1
C1728        1
C2170     

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Set the cutoff value to 1000 and create the list
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[
    application_df['CLASSIFICATION'].value_counts() < 1000
].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Confirm data types
application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [12]:
# Make a copy of application_df
application_df_copy = application_df.copy()

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
category_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 
                    'ORGANIZATION', 'INCOME_AMT']

application_df_copy = pd.get_dummies(application_df, columns=category_columns)

# Show updated dummies df
application_df_copy.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,True,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,True,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [14]:
# Split our preprocessed data into our features and target arrays
X = application_df_copy.drop(columns=['IS_SUCCESSFUL'])
y = application_df_copy['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def build_model(hp):
    nn = tf.keras.models.Sequential()
    
    # Add an Input layer
    nn.add(tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)))
    
    # First hidden layer
    nn.add(tf.keras.layers.Dense(
        units=hp.Int('units_1', min_value=50, max_value=150, step=10),
        activation='relu'
    ))
    
    # Second hidden layer
    nn.add(tf.keras.layers.Dense(
        units=hp.Int('units_2', min_value=20, max_value=100, step=10),
        activation='relu'
    ))
    
    # Output layer
    nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    nn.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', [0.001, 0.0005, 0.0001])
        ),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return nn


In [20]:
# Initialize the tuner
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=20,
    hyperband_iterations=2,
    directory='/app/data/processed/',  
    project_name='charity_tuning'    
)

In [21]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 21s]
val_accuracy: 0.728863000869751

Best val_accuracy So Far: 0.7294460535049438
Total elapsed time: 00h 08m 21s


In [23]:
# Display the best tuner params
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

{'units_1': 140,
 'units_2': 90,
 'learning_rate': 0.001,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [24]:
# Use the best hyperparameters to build the model
final_model = tuner.hypermodel.build(best_hps)

# Train the model with the training data
fit_model = final_model.fit(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7113 - loss: 0.5821 - val_accuracy: 0.7251 - val_loss: 0.5610
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7338 - loss: 0.5528 - val_accuracy: 0.7237 - val_loss: 0.5558
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7331 - loss: 0.5478 - val_accuracy: 0.7264 - val_loss: 0.5563
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7345 - loss: 0.5515 - val_accuracy: 0.7271 - val_loss: 0.5545
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7291 - loss: 0.5524 - val_accuracy: 0.7271 - val_loss: 0.5522
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7346 - loss: 0.5440 - val_accuracy: 0.7275 - val_loss: 0.5552
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7334 - loss: 0.5489 - val_accuracy: 0.7258 - val_loss: 0.5517
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7353 - loss: 0.5451 - val_accuracy: 0.

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = final_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 0s - 807us/step - accuracy: 0.7255 - loss: 0.5519
Loss: 0.5518904328346252, Accuracy: 0.7254810333251953


In [26]:
predictions = final_model.predict(X_test_scaled)
print(predictions)

268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step
[[0.6617508 ]
 [0.49320543]
 [0.7151472 ]
 ...
 [0.8016626 ]
 [0.33748746]
 [0.23996589]]


In [28]:
from sklearn.metrics import classification_report

# Convert probabilities into binary class predictions
predicted_classes = (predictions > 0.5).astype(int)

print(classification_report(y_test, predicted_classes))


              precision    recall  f1-score   support

           0       0.73      0.65      0.69      4003
           1       0.72      0.79      0.76      4572

    accuracy                           0.73      8575
   macro avg       0.73      0.72      0.72      8575
weighted avg       0.73      0.73      0.72      8575



In [29]:
# Export our model to HDF5 file
final_model.save('/app/data/processed/AlphabetSoupCharity_Optimization_2.h5')